In [1]:
import numpy as np
import pandas as pd
import scipy.sparse
import xgboost as xgb
import mix_pandas as mix
import predict as predict_mix
import db_column_name as db

%matplotlib inline
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'

from pylab import rcParams
rcParams['figure.figsize'] = 10, 7


cn = db.ColumnName()

target_minT = pd.read_csv('./data/31286_103.csv')
mix.set_index_date(target_minT, cn.date)

X = pd.read_csv('./data/character_31286.csv')
mix.set_index_date(X, cn.date)

X = X.drop([cn.point], axis=1)
X = X[[x for x in X.columns if 'avg' in x or 
       x == cn.offset]]
    
X = mix.mean_day(X)
target_minT.index = target_minT.index.round('D')

X = X.drop([cn.offset], axis=1)

target_minT = target_minT.reindex(X.index)
target_minT = mix.clean(target_minT)
X = X.reindex(target_minT.index)
X = mix.clean(X)

target_minT = mix.winsorized(target_minT, cn.value, [0.05, 0.95])
X = X.reindex(target_minT.index)
print(X.shape)


(1517, 54)


In [2]:
X = X[X.index.year < 2016]
target_minT = target_minT[target_minT.index.year < 2016]
print(X.shape)

(818, 54)


In [54]:
params = {
    'verbosity':0,
    'max_depth': 4,
}
reg = xgb.XGBRegressor(**params)

predict = predict_mix.predict_model_split(reg, X, target_minT, cn.value, 5)

thresholds = np.sort(reg.feature_importances_)
print(thresholds)

kbestThresh = thresholds[-6]
print(kbestThresh)

selection = SelectFromModel(reg, threshold=kbestThresh, prefit=True)
X_select = pd.DataFrame(selection.transform(X), index=X.index)
X_select.head()

[0.         0.         0.         0.         0.00067579 0.00075558
 0.00079248 0.00099232 0.00110264 0.00117776 0.00130059 0.0013006
 0.00137344 0.00156145 0.00158057 0.00160749 0.00163013 0.00173573
 0.00185956 0.00191419 0.00192625 0.00193021 0.00198481 0.00208002
 0.00211023 0.00221314 0.00225109 0.00225343 0.00226936 0.00230222
 0.00231661 0.00246244 0.00262455 0.00268345 0.00271371 0.00297993
 0.00317916 0.00334463 0.00369131 0.00408944 0.00423993 0.00424239
 0.00467458 0.00474179 0.0051435  0.00535087 0.00556024 0.00572591
 0.00636922 0.0063813  0.00791045 0.08708815 0.18167363 0.5981318 ]
0.0063692215


,0,1,2,3,4,5
2013-01-03,-28.59545,6.526540,-29.00020,-34.60150,-29.63050,-0.000013
2013-01-04,-26.42975,6.456630,-26.44405,-32.32505,-27.20305,-0.000036
2013-01-05,-29.51275,6.279280,-29.96565,-35.31215,-31.87540,-0.000008
2013-01-06,-31.34385,6.166875,-31.82275,-37.15400,-33.32260,0.000060
2013-01-08,-30.32600,5.352695,-31.20975,-35.51220,-31.78385,0.000041


In [56]:
params = {
    'verbosity':0,
    'max_depth': 4,
    
}
reg = xgb.XGBRegressor(**params)

predict = predict_mix.predict_model_split(reg, X_select, target_minT, cn.value, 5)
for train, test in predict:
    print("Train size {}".format(train.shape[0]))
    predict_mix.print_mean(train[[cn.value]], train[['prediction']])
    print("Test size {}".format(test.shape[0]))
    predict_mix.print_mean(test[[cn.value]], test[['prediction']])
    print()
    #     # 14.7 7.3 17.7 11.1 14.1

Train size 138
Mean squared error 0.19231
Mean absolute error 0.32696
Median absolute error 0.22778
Test size 136
Mean squared error 28.29137
Mean absolute error 4.55741
Median absolute error 4.40008

Train size 274
Mean squared error 0.82435
Mean absolute error 0.69177
Median absolute error 0.52891
Test size 136
Mean squared error 12.35535
Mean absolute error 2.70725
Median absolute error 2.23432

Train size 410
Mean squared error 1.77296
Mean absolute error 0.99055
Median absolute error 0.78188
Test size 136
Mean squared error 15.62670
Mean absolute error 2.93736
Median absolute error 2.16349

Train size 546
Mean squared error 2.48131
Mean absolute error 1.17017
Median absolute error 0.87809
Test size 136
Mean squared error 18.04887
Mean absolute error 3.04302
Median absolute error 2.04195

Train size 682
Mean squared error 3.52115
Mean absolute error 1.40836
Median absolute error 1.03034
Test size 136
Mean squared error 17.85061
Mean absolute error 3.40103
Median absolute error 2.83